# Notebook 11 - Select and Filter Meal Box Plans per Store

This notebook filters and ranks store-level meal box candidates by combining:

- Semantic recipe matches from earlier stages
- Store-specific product availability
- Waste-aware prioritization (waste and markdown signals)

We select only **fully matched recipes** where all required ingredients are available in a given store. The selected meal boxes are then enriched with ingredient lists to prepare them for final visualization or deployment.

Inputs:
- `store_mealbox_candidates.csv`: Candidate recipe–store pairs with average priority scores
- `store_recipe_matches.csv`: Store-ingredient availability matrix
- `recipes_with_ontology.csv`: Mapping from ingredients to recipes and concepts

Output:
- `store_mealbox_ranked.csv`: Final enriched meal box plan per store


In [1]:
import pandas as pd
import os

# Paths
input_folder = "cleaned_data"
output_folder = "cleaned_data"

# Load candidate boxes, store–ingredient matches, and recipe definitions
df_candidates = pd.read_csv(os.path.join(input_folder, "store_mealbox_candidates.csv"))
df_matches = pd.read_csv(os.path.join(input_folder, "store_recipe_matches.csv"))
df_recipes = pd.read_csv(os.path.join(input_folder, "recipes_with_ontology.csv"))

print("Candidates:", df_candidates.shape)
print("Matches:", df_matches.shape)
print("Recipes:", df_recipes.shape)


Candidates: (9, 3)
Matches: (54, 4)
Recipes: (6, 6)


In [3]:
# Patch: Add recipe column to matches if missing
if "recipe" not in df_available.columns:
    df_available = df_available.merge(df_recipes[["ingredient", "recipe"]], on="ingredient", how="left")

# Group ingredients per recipe per store
ingredient_lists = (
    df_available.groupby(["store", "recipe"])["ingredient"]
    .apply(list)
    .reset_index()
)

# Merge back into candidate ranking
df_ranked = df_candidates.merge(ingredient_lists, on=["store", "recipe"], how="left")

# Drop rows with missing ingredients
df_ranked = df_ranked[df_ranked["ingredient"].notna()]

# Rename for clarity
df_ranked.rename(columns={"ingredient": "ingredients"}, inplace=True)

# Preview result
display(df_ranked.head())


,store,recipe,avg_priority,ingredients
0,1024,Greek Yogurt & Honey,1,[yogurt]
1,1058,Greek Yogurt & Honey,1,[yogurt]
2,1160,Greek Yogurt & Honey,1,[yogurt]
3,3298,Greek Yogurt & Honey,1,[yogurt]
4,3345,Greek Yogurt & Honey,1,[yogurt]


In [4]:
# Final export
output_file = os.path.join(output_folder, "store_mealbox_ranked.csv")
df_ranked.to_csv(output_file, index=False)

print("-> Saved ranked store-level meal boxes to:", output_file)


-> Saved ranked store-level meal boxes to: cleaned_data\store_mealbox_ranked.csv


### Summary

This notebook finalizes the store-specific meal box recommendation output by:

- Reconstructing per-store ingredient lists for each complete recipe
- Filtering out incomplete or untraceable matches
- Renaming and formatting fields for clarity
- Exporting the enriched meal box structure for operational use

This ranked output supports **inventory planning** and **meal kit creation** across store branches and will be used for final visualization and evaluation in Notebook 12.
